In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.tsa.statespace.varmax import VARMAX

In [3]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.style.use('seaborn-dark-palette')
plt.rcParams["mathtext.fontset"] = "cm"

In [4]:
def num_chamber(list_x, list_y):
    list_x = list_x.values
    list_y = list_y.values
    list_c = list()
    for i in range(len(list_x)):
        x = int(list_x[i])
        y = int(list_y[i])
        if (y > 178):
            list_c.append(5)
        else:
            if ((y <= 0)&(x <= 0)):
                list_c.append(5)
            else:
                if (y <= 46):
                    list_c.append(1)
                else:
                    if (y <= 92):
                        list_c.append(2)
                    else:
                        if (y <= 138):
                            list_c.append(3)
                        else:
                            list_c.append(4)
    return list_c  

In [5]:
def plot_time_series(ts_1, ts_label_1, ts_2, ts_label_2, title, path):
    assert len(ts_1) == len(ts_2)
    xs = list(range(0, len(ts_1)))

    plt.plot(xs, ts_1, c='green', label=ts_label_1)
    plt.plot(xs, ts_2, c='red', label=ts_label_2)

    plt.title(title)
    plt.legend(loc='upper left')
    plt.savefig(path)
    plt.show()

In [ ]:
# Data Loading
data_df = pd.read_csv('../dataset/ant/location_in_mm.csv')
c = [1, 74, 154]
amount_ant = [73, 80, 83]
pred = pd.DataFrame()
for colony_id in range(1, 4):

    for ant_id in range(amount_ant[colony_id-1]):
        #ant_id = 0
        colony = c[colony_id-1]
        data = data_df[(data_df['colony_id'] == colony_id) & (data_df['ant_id'] == ant_id)]

        num_ant = colony + ant_id
        data.reset_index(inplace = True)
        data = data[['location_x', 'location_y']]

        #creating the train and validation set
        valid = data[int(0.5*(len(data))):]
        train = data[:int(0.5*(len(data)))]

        tmp = []
        for p in tqdm(range(5)):
            for q in tqdm(range(5)):
                try:
                    tmp.append([VARMAX(train, order = (p,q)).fit().bic, p, q])
                except:
                    tmp.append([None, p, q])
        tmp = pd.DataFrame(tmp,columns = ['bic', 'p', 'q'])
        print(tmp)
        order = tmp[tmp['bic'] == tmp['bic'].min()]
        print(order)

        try: 
            model = VARMAX(train, order = (order['p'], order['q']))
            fitted_model = model.fit()
            prediction = fitted_model.forecast(len(valid)).reset_index(drop=True) 
            prediction = pd.DataFrame(prediction)
            prediction = pd.concat([train, prediction], axis = 0)

            prediction_c = num_chamber(prediction['location_x'], prediction['location_y'])
            pred['%i'%num_ant] = pd.DataFrame(prediction_c, columns = ['%i'%num_ant])
            

            #plot_time_series(ts_1 = prediction_chamber, ts_label_1 = 'VARMA Model', ts_2 = valid['chamber'], ts_label_2 = 'Close', title = 'VARMA predictions vs. ground truth')
            plot_time_series(ts_1 = prediction['location_x'], ts_label_1 = 'VARMA Model', ts_2 = data['location_x'], ts_label_2 = 'Close', title = 'VARMA predictions vs. ground truth of location x', path = '../figures/insect/%i/x1.png'%num_ant)
            plot_time_series(ts_1 = prediction['location_y'], ts_label_1 = 'VARMA Model', ts_2 = data['location_y'], ts_label_2 = 'Close', title = 'VARMA predictions vs. ground truth of location y', path = '../figures/insect/%i/y1.png'%num_ant)

        except Exception as reason:
            print('%i'%num_ant, reason)
            
prediction_c.to_csv('../dataset/ant/predictions_VARMAX_noloop/pred_VARMA_noloop.csv', index = False)   

  0%|          | 0/5 [00:00<?, ?it/s]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 40%|████      | 2/5 [00:02<00:03,  1.33s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 60%|██████    | 3/5 [00:05<00:04,  2.13s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 80%|████████  | 4/5 [00:11<00:03,  3.60s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarni

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 40%|████      | 2/5 [00:11<00:18,  6.29s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 60%|██████    | 3/5 [00:21<00:15,  7.78s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,

             bic  p  q
0            NaN  0  0
1   15671.107918  0  1
2   15245.905377  0  2
3   15056.504522  0  3
4   14460.943291  0  4
5    4106.346745  1  0
6    4101.476918  1  1
7    4063.999851  1  2
8    4080.088121  1  3
9    4094.833438  1  4
10   4090.855475  2  0
11   4069.893763  2  1
12   4088.261315  2  2
13   4108.444990  2  3
14   4089.727433  2  4
15   4056.071008  3  0
16   4064.456208  3  1
17   4081.596128  3  2
18   4096.821962  3  3
19   4101.018151  3  4
20   4062.764338  4  0
21   4079.424850  4  1
22   4093.877794  4  2
23   4055.425621  4  3
24   4067.169156  4  4
            bic  p  q
23  4055.425621  4  3


/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  0%|          | 0/5 [00:00<?, ?it/s]

1 Cannot set a frame with no defined index and a value that cannot be converted to a Series


...


 60%|██████    | 3/5 [00:12<00:09,  4.72s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 80%|████████  | 4/5 [00:21<00:06,  6.48s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: Conve

             bic  p  q
0            NaN  0  0
1   12766.952110  0  1
2   12427.569810  0  2
3   11996.845753  0  3
4   10501.615247  0  4
5   14059.291678  1  0
6   11628.614220  1  1
7   10644.603772  1  2
8   11255.036752  1  3
9   10343.278158  1  4
10   1641.007349  2  0
11   1616.866636  2  1
12   1439.881931  2  2
13   1408.582644  2  3
14   1207.532893  2  4
15   1570.943929  3  0
16   1590.384477  3  1
17   1366.002301  3  2
18   1362.954473  3  3
19   1157.486035  3  4
20  13109.839359  4  0
21  11324.152457  4  1
22  12776.516919  4  2
23  11206.498573  4  3
24  10623.520789  4  4
            bic  p  q
19  1157.486035  3  4


/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  0%|          | 0/5 [00:00<?, ?it/s]

10 Cannot set a frame with no defined index and a value that cannot be converted to a Series


/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 40%|████      | 2/5 [00:01<00:02,  1.36it/s]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 60%|██████    | 3/5 [00:03<00:02,  1.35s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 80%|████████  | 4/5 [00:06<00:02,  2.06s/it]/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization f